# Self-evaluation
In this notebook we prompt a model to describe an image and then evaluate the description.

This involves using an image as a [scenario](https://docs.expectedparrot.com/en/latest/scenarios.html) of survey questions, and [piping](https://docs.expectedparrot.com/en/latest/notebooks/piping_comments.html) the answer to a question into another question.

We also show how to use [FileStore](https://docs.expectedparrot.com/en/latest/filestore.html) to post and retrieve files to [Coop](https://docs.expectedparrot.com/en/latest/coop.html) for ease of sharing data and other content used with surveys.

Please see the [docs](https://docs.expectedparrot.com) for more examples and details on all of these methods!

In [1]:
from edsl import QuestionFreeText, Survey, Scenario, Model, FileStore

In [2]:
m = Model("gemini-1.5-flash")

In [3]:
q1 = QuestionFreeText(
    question_name = "describe",
    question_text = "Describe this image: {{ scenario.image }}"
)

In [4]:
q2 = QuestionFreeText(
    question_name = "improvements",
    question_text = """
    Consider the following image and description of it.
    In what ways could this description be improved?
    Image: {{ scenario.image }}
    Description: {{ describe.answer }}
    """
)

In [5]:
survey = Survey([q1,q2])

Posting an image to Coop and retrieving it to use as a scenario:

In [6]:
# creating a FileStore object
fs = FileStore("parrot_logo.png", visibility = "public") 

# posting it to Coop
info = fs.push(visibility = "public") 
info

{'description': 'File: parrot_logo.png',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/3a3cd3f2-fcea-4d25-b4e4-25d564ee7a3b',
 'uuid': '3a3cd3f2-fcea-4d25-b4e4-25d564ee7a3b',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

In [7]:
# retrieving it
file = FileStore.pull(info["uuid"])

# creating a scenario
s = Scenario({
    "image":file,
    "image_id":"parrot" # metadata for results
})
s

,key,value
0,image:path,parrot_logo.png
1,image:base64_string,...
2,image:binary,True
3,image:suffix,png
4,image:mime_type,image/png
5,image:external_locations,{}
6,image:extracted_text,None
7,image_id,parrot


In [8]:
results = survey.by(s).by(m).run()

Job UUID,5e1ac5b4-92d9-4916-a3e6-d4cadc878468
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/5e1ac5b4-92d9-4916-a3e6-d4cadc878468
Exceptions Report URL,None
Results UUID,7ab34dbd-87cd-4cb1-a156-1a0644b51a8c
Results URL,https://www.expectedparrot.com/content/7ab34dbd-87cd-4cb1-a156-1a0644b51a8c


In [9]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.describe
4,answer.improvements
5,cache_keys.describe_cache_key
6,cache_keys.improvements_cache_key
7,cache_used.describe_cache_used
8,cache_used.improvements_cache_used
9,comment.describe_comment


In [10]:
results.select("model", "image_id", "describe", "improvements")

,model.model,scenario.image_id,answer.describe,answer.improvements
0,gemini-1.5-flash,parrot,"The image shows a close-up shot of a vibrant, multicolored macrame wall hanging. The macrame is densely woven, creating a textured and somewhat three-dimensional effect. The colors are predominantly earthy tones, including various shades of brown, beige, cream, and perhaps some muted greens or yellows, though the exact shades are difficult to determine precisely due to the lighting and image resolution. The strands are relatively thick, giving the piece a substantial and rustic feel. The overall shape is irregular, lacking a perfectly geometric form, contributing to its organic and handcrafted appearance. The background is blurred, drawing focus solely to the macrame itself. The lighting is soft and diffused, preventing harsh shadows and highlighting the intricate knotting.","The description is quite good, but here are some ways it could be improved: **1. More Specific and Evocative Language:** * Instead of ""various shades of brown, beige, cream, and perhaps some muted greens or yellows,"" try to be more specific. For example: ""rich chocolate browns, creamy ivories, sandy beiges, with hints of sage green and mustard yellow peeking through."" Using more evocative adjectives will make the description more engaging. * Instead of ""somewhat three-dimensional effect,"" describe the texture more vividly. For example: ""The densely woven strands create a richly textured surface, with layers of knots and loops that give it depth and a pleasingly uneven topography."" * Instead of ""irregular shape,"" describe the *kind* of irregularity. Is it organically flowing? Does it have a particular shape that resembles something (e.g., a sunburst, a cascade)? For example: ""The overall shape is organically asymmetrical, cascading downwards in a pleasingly irregular pattern."" **2. Sensory Details:** * The description focuses heavily on visual aspects. Consider adding other sensory details if possible. For instance, could you infer anything about the texture from the image? Would it feel soft, rough, coarse? This would add another layer of engagement. **3. Emotional Response:** * The description is factual, but could benefit from conveying the emotional impact of the image. Does the macrame evoke a feeling of warmth, calmness, rustic charm, bohemian style, etc.? Adding a sentence or two about the overall feeling the piece inspires would make it more compelling. For example: ""The piece exudes a sense of handcrafted warmth and bohemian charm."" **4. Size and Scale:** * Adding information about the approximate size of the macrame would give the reader a better sense of scale. Is it a small wall hanging or a large, statement piece? **5. Style and Technique:** * If you can identify the specific macrame knotting techniques used (e.g., square knots, spiral knots, etc.), mentioning them would add to the description's accuracy and sophistication. **Example of Improved Description:** ""This close-up shot showcases a vibrant macrame wall hanging, approximately 24 inches wide, densely woven with rich chocolate browns, creamy ivories, sandy beiges, and subtle hints of sage green and mustard yellow. The densely packed strands create a richly textured surface, with layers of intricate square and spiral knots that give it considerable depth and a pleasingly uneven topography. The piece cascades downwards in an organically asymmetrical pattern, reminiscent of a sunburst or a gently flowing waterfall. The thick cords lend a substantial, rustic feel, while the soft, diffused lighting highlights the intricate knotting without harsh shadows. The blurred background keeps the focus firmly on the macrame itself, which exudes a sense of handcrafted warmth and bohemian charm. The overall effect is one of relaxed elegance and artistic skill."" By incorporating these suggestions, you can transform a good description into a truly captivating and evocative one. Remember to tailor 

### Posting this notebook to Coop

In [11]:
from edsl import Notebook

n = Notebook("content_evaluation_example.ipynb")
n.push(
    description = "Image scenario example", 
    alias = "content-evaluation-example-notebook", 
    visibility = "public"
)

{'description': 'Image scenario example',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/fa9ce40a-e0de-46d8-8b76-11d9d4780cb3',
 'uuid': 'fa9ce40a-e0de-46d8-8b76-11d9d4780cb3',
 'version': '0.1.47.dev1',
 'visibility': 'public'}